In [19]:
import os
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from datetime import date
import statsmodels.api as sm
from matplotlib import dates
from IPython.core.display import display, HTML
import matplotlib.ticker as mtick
from matplotlib.ticker import MaxNLocator
import itertools
from dateutil.relativedelta import relativedelta
from scipy import stats

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

from datetime import datetime
import sys

warnings.filterwarnings('ignore')
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import snowflake.connector
cnx = snowflake.connector.connect(
    user='nmittleman@toasttab.com',
    account='toast.us-east-1',
    authenticator='externalbrowser'
    )

%matplotlib inline

def QueryDWH(query, cnx, col=[]):
    cur = cnx.cursor()
    cur.execute(query)
    # If there is no col input, then get the columns from the return of the
    # snowflake query
    if not col:
        df = pd.DataFrame(cur.fetchall())
        if not df.empty:
            df.columns = [desc[0] for desc in cur.description]
    else:
        df = pd.DataFrame(cur.fetchall(), columns=col)

    return df
QueryDWH("""USE DATABASE TOAST""", cnx)
QueryDWH("""USE WAREHOUSE TOAST_WH""", cnx)


def get_quarter(date):
    return int((date.month - 1) / 3) + 1

def get_first_day_of_the_quarter(date):
    quarter = get_quarter(date)
    return datetime(date.year, int(3 * quarter), 1)
database = 'toast'
# database = 'toast_clone_20220121'

In [20]:
## Live ARR 
query = f"""SELECT 
                   -- CONCAT('Q', QUARTER(start_date) ,'-' ,  YEAR(start_date)) AS period_name
                 -- , DATE_TRUNC('Quarter', start_date) AS start_date
                 period_name
                 , start_date
                 , end_date
                 
                 , live_core_payments_arr
                 , live_guest_payments_arr
                 , live_payments_arr
                 
                 , live_core_saas_arr
                 , live_guest_sfdc_saas_arr
                 , total_rev_share AS "Live Rev Share ARR"
                 , live_ec_saas_arr AS "Live EC ARR"                     
                 , live_xtrachef_saas_arr

                 , live_saas_arr AS "Live SaaS ARR"
                 
                 
                 
                 
                 , live_arr AS "Total Live ARR"
            FROM {database}.key_metrics.key_metrics
            WHERE period_type = 'FULL MONTH'
            ORDER BY 2 DESC"""
arr = QueryDWH(query, cnx, ['Month', 'Month Start', 'Month End', 'Live Core Payments ARR', 'Live Guest Payments ARR', 'Live Payments ARR', 'Live Core SaaS ARR', 'Live Guest SFDC SaaS ARR', 'Live Rev Share ARR', 'Live EC ARR', 'Live XC ARR', 'Live SaaS ARR',
                           'Live ARR' ])


query = f"""
            SELECT --DATE_TRUNC('Quarter',firstdayofmonth) AS start_date
                 b.firstdayofmonth
                 , DAYSINMONTH
                 , SUM(transaction_volume) / DAYSINMONTH * 365 AS gpv
                 , SUM(transaction_count) / DAYSINMONTH * 365 AS gpv_tx
                 , SUM(transaction_volume) AS gpv
                 , SUM(transaction_count) AS gpv_tx                 
            FROM {database}.payments.directional_payments_basic a
            INNER JOIN {database}.analytics_core.date_dim_underlying b
            ON a.processor_date_id = b.date_id
            GROUP BY 1,2;
            """
gpv = QueryDWH(query, cnx, ['Month Start', 'DaysInMonth', 'Annualized GPV', 'Annualized GPV Transactions', 'Monthly GPV', 'Monthly GPV Transactions'])
gpv.drop(['DaysInMonth'], axis=1, inplace=True, errors='ignore')

## Live POS Customers
query = f"""SELECT 
             --   CONCAT('Q', QUARTER(start_date) ,'-' ,  YEAR(start_date)) AS period_name
             -- , DATE_TRUNC('Quarter', start_date) AS start_date
              period_name
             , start_date
             , end_date
             , TOTAL_POS_WINS
             , POS_GO_LIVES
             , live_pos_customers
             , NPS_SCORE
        FROM {database}.key_metrics.key_metrics
        WHERE period_type = 'FULL MONTH'
        -- AND  end_date = LAST_DAY(start_date, 'Quarter')
        ORDER BY 2 DESC"""
cust_count = QueryDWH(query, cnx, ['Month', 'Month Start', 'Month End', 'Total Bookings', 'POS Go Lives','Live POS Customers',   'NPS'])



In [21]:

query = f"""
        SELECT DATE_TRUNC('Month', dt) AS  month
             , COUNT(DISTINCT IFF((DATE_TRUNC('Month', nbo_churn_date) = DATE_TRUNC('Month', dt)) AND (pos_first_order_date IS NOT NULL) , salesforce_accountid, NULL) ) AS churns
             , COUNT(DISTINCT IFF((DATE_TRUNC('Month', nbo_churn_date) = DATE_TRUNC('Month', dt)) AND (pos_first_order_date IS NULL) , salesforce_accountid, NULL) ) AS cancelled_contracts
        FROM {database}.payments.daily_customer_details
        GROUP BY 1
        """
churn = QueryDWH(query, cnx, ['Month Start', 'Post-Live Churns', 'Canceled Contracts'])

query = f"""
        WITH first_account_closed AS 
        (SELECT b.highest_parent_id
             , MIN(DATE_TRUNC('Month', a.nbo_close_date)) AS first_location_booked_month
        FROM {database}.payments.daily_customer_details a
        INNER JOIN {database}.analytics_core.account_segment b
        ON a.customer_id = b.account_id
        WHERE a.nbo_is_won
        GROUP BY 1)
        SELECT first_location_booked_month
             , COUNT(highest_parent_id)
        FROM first_account_closed
        GROUP BY 1
        """
new_accounts = QueryDWH(query, cnx, ['Month Start', 'New Booked Accounts'])
new_accounts['Month Start'] = pd.to_datetime(new_accounts['Month Start'] ).dt.date

query = f"""
        SELECT DATE_TRUNC('Month', dt) AS  month
             , COUNT(DISTINCT IFF(pos_status IN ('Live', 'Backlog'), b.highest_parent_id, NULL)) AS booked_accounts
             , COUNT(DISTINCT IFF(pos_status IN ('Live'), b.highest_parent_id, NULL)) AS live_accounts
        FROM {database}.payments.daily_customer_details a
        INNER JOIN {database}.analytics_core.account_segment b
        ON a.customer_id = b.account_id
        WHERE dt = LAST_DAY(dt)
        GROUP BY 1
        """
live_accounts = QueryDWH(query, cnx, ['Month Start', 'Booked POS Accounts', 'Live POS Accounts'])
live_accounts['Month Start'] = pd.to_datetime(live_accounts['Month Start'] ).dt.date

In [22]:
query = """
  WITH cust_module AS
            (SELECT  customer_id
                   , DATE_TRUNC('Quarter', dt) AS quarter
                      , SUM(IFF(product_name IN ('Software Monthly Subscription',
                                              'Additional Tablet Monthly Software Subscription',
                                              'Caller ID Monthly Subscription'), saas_quantity, 0)) AS core_count
                   , SUM(IFF(product_name IN ('Enterprise Data Management Monthly Subscription',
                                              'Enterprise Software Monthly Subscription'), saas_quantity, 0)) AS ent_count 
                   , SUM(IFF(product_name IN ('Handheld Monthly Software Subscription'), saas_quantity, 0)) AS hh_count
                   , SUM(IFF(product_name IN ('Online Ordering Monthly Subscription'), saas_quantity, 0)) AS oo_count
                   , SUM(IFF(product_name IN ('Gift Card Program Monthly Subscription'), saas_quantity, 0)) AS gc_count
                   , SUM(IFF(product_name IN ('API Monthly Subscription'), saas_quantity, 0)) AS api_count
                   , SUM(IFF(product_name IN ('Loyalty Program Monthly Subscription'), saas_quantity, 0)) AS loy_count
                   , SUM(IFF(product_name IN ('Marketing Monthly Subscription - 1st Location', 'Marketing Monthly Subscription - Additional Location', 'CRM Monthly Subscription'), saas_quantity, 0)) AS marketing_count
                   , SUM(IFF(product_name IN ('Toast Order & Pay', 'Toast Mobile Order & Pay'), saas_quantity, 0)) AS opt_count
                   , SUM(IFF(product_name IN ('Kitchen Display Screen Monthly Subscription'), saas_quantity, 0)) AS kds_count
                   , SUM(IFF(product_name IN ('Kiosk Monthly Subscription'), saas_quantity, 0)) AS kiosk_count
                   , SUM(IFF(product_name IN ('Inventory Monthly Subscription'), saas_quantity, 0)) AS inv_count
               , SUM(IFF(product_name IN ('xtraCHEF Pro' , 'Invoice Automation', 'Recipe Costing', 'Inventory Management'), saas_quantity, 0)) AS xc_count
               , SUM(IFF(product_name IN ('DoorDash Integration', 'GrubHub Integration', 'UberEats Integration') AND dt >= '2021-04-01', saas_quantity, 0)) AS three_pd_count
               
               ,  IFF(ent_count > 0, 1, 0) + IFF(hh_count > 0, 1, 0) + IFF(oo_count > 0, 1, 0) + IFF(gc_count > 0, 1, 0) 
                + IFF(api_count > 0, 1, 0) + IFF(loy_count > 0, 1, 0) + IFF(marketing_count > 0, 1, 0) + IFF(kds_count > 0, 1, 0) 
                + IFF(opt_count > 0, 1, 0) + IFF(kiosk_count > 0, 1, 0) + IFF(inv_count > 0, 1, 0) + IFF(three_pd_count > 0, 1, 0)
                + IFF(xc_count > 0, 1, 0) AS module_count_live_today
              FROM analytics_core_arr.saas_daily_status
              WHERE LAST_DAY(dt, 'Quarter') = dt
              AND dt >= '2018-01-01'
              AND pos_status = 'Live'
              AND saas_status = 'Live'
              GROUP BY 1,2), tds AS 
              (SELECT DATE_TRUNC('Quarter', dt) AS quarter
                    , customer_id
                    , SUM(TDS_TRANSACTIONS) AS tds_tx
                    , IFF(tds_tx > 0, 1, 0) AS tds_count
                 FROM payments.daily_customer_details 
                 WHERE LAST_DAY(dt, 'Quarter') = LAST_DAY(dt, 'Month')
                 GROUP BY 1,2
               ), ec AS 

              (        SELECT DISTINCT
                        DATE_TRUNC('Quarter', p.check_date) AS quarter
                      , t.customer_id
                      , 1 AS ec
                    FROM source_estratex.payroll_earning_historical_entity_current e
                    LEFT JOIN source_estratex.payroll_historical_entity_current p
                        ON e.estratex_payroll_uuid = p.estratex_payroll_uuid
                    LEFT JOIN source_estratex.location_entity_current l
                        ON e.estratex_location_uuid = l.estratex_location_uuid
                    INNER JOIN analytics_core.toastorders_customer_bridge t
                    ON l.restaurant_guid = t.toastorders_restaurant_guid
                    WHERE DATE_TRUNC(month, p.check_date) >= '2018-01-01'),

              live_count AS
            (SELECT  DATE_TRUNC('Quarter', dt) AS quarter
                   , customer_id
                   , 1 AS live_locs
              FROM payments.daily_customer_details
              WHERE LAST_DAY(dt, 'Quarter') = dt
              AND dt >= '2018-01-01'
              AND pos_status = 'Live'), cust_mod_count AS   
            (SELECT c.customer_id
                 , c.quarter
                 , ZEROIFNULL(b.tds_count) AS  tds_count
                 , a.core_count
                 , a.ent_count 
                 , a.hh_count
                 , a.oo_count
                 , a.gc_count
                 , a.api_count
                 , a.loy_count
                 , a.marketing_count
                 , a.opt_count
                 , a.kds_count
                 , a.kiosk_count
                 , a.inv_count
                 , a.three_pd_count
                 , a.xc_count
                 , ZEROIFNULL(d.ec) AS ec_count
                 , a.module_count_live_today + ZEROIFNULL(b.tds_count) + ZEROIFNULL(d.ec) AS unique_mod
            FROM live_count c
            LEFT JOIN cust_module a
            ON a.quarter = c.quarter
            AND a.customer_id = c.customer_id
            LEFT JOIN tds b
            ON a.quarter = b.quarter
            AND a.customer_id = b.customer_id
            LEFT JOIN ec d
            ON a.quarter = d.quarter
            AND a.customer_id = d.customer_id)
            SELECT quarter
                 , COUNT(customer_id) AS ct
                 , COUNT(DISTINCT IFF(unique_mod = 0, customer_id, NULL)) AS zero
                 , COUNT(DISTINCT IFF(unique_mod = 1, customer_id, NULL)) AS one
                 , COUNT(DISTINCT IFF(unique_mod = 2, customer_id, NULL)) AS two
                 , COUNT(DISTINCT IFF(unique_mod = 3, customer_id, NULL)) AS three
                 , COUNT(DISTINCT IFF(unique_mod = 4, customer_id, NULL)) AS four                
                 , COUNT(DISTINCT IFF(unique_mod = 5, customer_id, NULL)) AS five                 
                 
                 , COUNT(DISTINCT IFF(unique_mod >= 3, customer_id, NULL)) AS three_plus
                 , COUNT(DISTINCT IFF(unique_mod >= 4, customer_id, NULL)) AS four_plus
                 , COUNT(DISTINCT IFF(unique_mod >= 5, customer_id, NULL)) AS five_plus                 
                 , COUNT(DISTINCT IFF(unique_mod >= 6, customer_id, NULL)) AS six_plus
                 
                 , zero / ct * 100 AS zero_pct
                 , one / ct * 100 AS one_pct
                 , two / ct * 100 AS two_pct
                 , three / ct * 100 AS three_pct                 
                 , four / ct * 100 AS four_pct
                 , five / ct * 100 AS five_pct
                 
                 , three_plus / ct * 100 AS three_plus_pct
                 , four_plus / ct * 100 AS four_plus_pct
                 , five_plus / ct * 100 AS five_plus_pct
                 , six_plus / ct * 100 AS six_plus_pct
                 
            FROM cust_mod_count
            GROUP BY 1
            ORDER BY 1 DESC
        """

# Looks at number of customers with 3+,4+,5+ modules live in a time period
better_together = QueryDWH(query, cnx)

In [23]:
np.round(better_together, 2)

,QUARTER,CT,THREE_PLUS,FOUR_PLUS,FIVE_PLUS,THREE_PLUS_PCT,FOUR_PLUS_PCT,FIVE_PLUS_PCT
0,2022-01-01,61814,47613,37368,27415,77.026200,60.452300,44.350800
1,2021-10-01,56644,43013,33265,23940,75.935700,58.726400,42.264000
2,2021-07-01,52080,38867,29706,21485,74.629400,57.039200,41.253800
3,2021-04-01,47942,35104,26360,18523,73.221800,54.983100,38.636300
4,2021-01-01,43269,30767,22197,14868,71.106300,51.300000,34.361800
5,2020-10-01,40108,27748,19393,12491,69.183200,48.351900,31.143400
6,2020-07-01,36771,24505,16806,10460,66.642200,45.704500,28.446300
7,2020-04-01,33129,21623,14603,8800,65.269100,44.079200,26.562800
8,2020-01-01,30421,18874,12301,7146,62.042700,40.435900,23.490400
9,2019-10-01,26826,15727,9943,5619,58.626000,37.064800,20.946100


## ARR

In [24]:
arr.head()

,Month,Month Start,Month End,Live Core Payments ARR,Live Guest Payments ARR,Live Payments ARR,Live Core SaaS ARR,Live Guest SFDC SaaS ARR,Live Rev Share ARR,Live EC ARR,Live XC ARR,Live SaaS ARR,Live ARR
0,April - 2022,2022-04-01,2022-04-17,305286836.7437,48886884.3659,354173721.1096,191492075.3767,75088177.7640,5639073.3600,26964282.9254,6882776.1600,306066385.5861,660240106.6957
1,March - 2022,2022-03-01,2022-03-31,285529832.5286,47706668.3750,333236500.9036,187172036.9711,73093169.4094,9642836.7600,26604976.5282,6770371.6800,303283391.3487,636519892.2523
2,February - 2022,2022-02-01,2022-02-28,256874983.0009,47915713.2495,304790696.2504,178072760.7252,68656067.4811,7858243.4400,23747076.9786,6938495.7600,285272644.3849,590063340.6353
3,January - 2022,2022-01-01,2022-01-31,244223747.9261,47253279.4548,291477027.3809,171315832.4279,64986131.8941,7958948.5200,22335402.1200,6368854.8000,272965169.7621,564442197.1429
4,December - 2021,2021-12-01,2021-12-31,254214307.5468,48171940.2958,302386247.8426,168137276.8934,62755802.3999,6938719.3200,21086678.1600,6256898.4000,265175375.1732,567561623.0159


In [25]:
1695362.40 / 12

141280.19999999998

In [26]:
# Reformat the ARR data and rename into more visually pleasant columns
cols = arr.columns.tolist()
cols.remove('Month')
cols.remove('Month Start')
cols.remove('Month End')

for col in cols:
    arr[col] = arr[col].astype(float)
    arr[col].fillna(0, inplace=True)



arr = arr[['Month', 'Month Start', 'Month End', 
           'Live Core Payments ARR', 'Live Guest Payments ARR', 'Live Payments ARR',
           'Live Core SaaS ARR', 'Live Guest SFDC SaaS ARR', 'Live Rev Share ARR', 'Live EC ARR', 'Live XC ARR', 'Live SaaS ARR',
           'Live ARR']]
arr[pd.to_datetime(arr['Month']).dt.date < date(2022,1,1)][['Month', 'Live Rev Share ARR','Live SaaS ARR','Live Payments ARR','Live ARR']].head(24)

,Month,Live Rev Share ARR,Live SaaS ARR,Live Payments ARR,Live ARR
4,December - 2021,6938719.32,2.651754e+08,3.023862e+08,5.675616e+08
5,November - 2021,5962192.56,2.542102e+08,3.029552e+08,5.571654e+08
6,October - 2021,5758666.68,2.421121e+08,3.125012e+08,5.546134e+08
7,September - 2021,1695362.40,2.262200e+08,3.175640e+08,5.437840e+08
8,August - 2021,1732376.16,2.156836e+08,3.176471e+08,5.333308e+08
9,July - 2021,1734990.84,2.060790e+08,3.156751e+08,5.217541e+08
10,June - 2021,1733153.76,1.961478e+08,2.983099e+08,4.944577e+08
11,May - 2021,1926319.32,1.824929e+08,2.888320e+08,4.713249e+08
12,April - 2021,1966822.56,1.696306e+08,2.529835e+08,4.226141e+08
13,March - 2021,1968134.76,1.605731e+08,2.233161e+08,3.838892e+08


# Customers Count

In [27]:
# Combines a few data sets to the total customer, location, churn and new customer counts by month
cust_count['Month Start'] = pd.to_datetime(cust_count['Month Start']).dt.date
cust_count['Month End'] = pd.to_datetime(cust_count['Month End']).dt.date

cust_count_with_churn = cust_count.merge(churn, how='left').merge(new_accounts, how='left').merge(live_accounts, how='left')
cust_count_with_churn = cust_count_with_churn[['Month Start', 'Month End', 'Total Bookings', 'New Booked Accounts', 'Canceled Contracts', 'POS Go Lives', 'Post-Live Churns', 'Live POS Customers',  'Booked POS Accounts', 'Live POS Accounts','NPS']]
cust_count_with_churn.head()

,Month Start,Month End,Total Bookings,New Booked Accounts,Canceled Contracts,POS Go Lives,Post-Live Churns,Live POS Customers,Booked POS Accounts,Live POS Accounts,NPS
0,2022-04-01,2022-04-17,989,596,143,1166.0,260,62713,NaN,NaN,29.9840
1,2022-03-01,2022-03-31,3655,2028,357,2359.0,418,61814,43862.0,36547.0,25.5309
2,2022-02-01,2022-02-28,2487,1382,328,2079.0,362,59887,42212.0,35473.0,23.3023
3,2022-01-01,2022-01-31,2228,1206,909,1928.0,372,58190,41164.0,34536.0,29.2430
4,2021-12-01,2021-12-31,3492,1614,194,1784.0,379,56644,40653.0,33656.0,32.0855


## Volume

In [28]:
# gmv['Month Start'] = pd.to_datetime(gmv['Month Start']).dt.date
gpv['Month Start'] = pd.to_datetime(gpv['Month Start']).dt.date
volume = arr[['Month', 'Month Start', 'Month End']].merge(gpv)
volume

,Month,Month Start,Month End,Annualized GPV,Annualized GPV Transactions,Monthly GPV,Monthly GPV Transactions
0,April - 2022,2022-04-01,2022-04-17,49969943667.10166545,1077700654.666545,4107118657.57,88578136
1,March - 2022,2022-03-01,2022-03-31,83063614457.78225830,1836145698.870850,7054717940.25,155946621
2,February - 2022,2022-02-01,2022-02-28,75478357443.39285610,1640706337.857195,5790120571.00,125862404
3,January - 2022,2022-01-01,2022-01-31,57919059425.23225830,1328115032.742065,4919152992.28,112798811
4,December - 2021,2021-12-01,2021-12-31,69713990741.09032105,1447352702.903320,5920914282.12,122925846
5,November - 2021,2021-11-01,2021-11-30,64877485814.10500000,1457519703.333455,5332396094.31,119796140
6,October - 2021,2021-10-01,2021-10-31,68696030169.95161255,1575922587.096680,5834457356.90,133845480
7,September - 2021,2021-09-01,2021-09-30,64063662549.40500000,1502066761.000000,5265506510.91,123457542
8,August - 2021,2021-08-01,2021-08-31,64007942915.28387085,1482249458.064575,5436291042.12,125889680
9,July - 2021,2021-07-01,2021-07-31,68488906418.91290405,1545667007.096680,5816866024.62,131275828


# MRR

In [29]:
output_folder = os.path.join('./',f'Pitch Deck Outputs {str(datetime.now().date())}')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(os.path.join(output_folder,"pitch_deck_metrics.xlsx"), engine='xlsxwriter')

# Write each dataframe to a different worksheet.
arr.to_excel(writer, sheet_name='ARR', index=False)
cust_count_with_churn.to_excel(writer, sheet_name='Customer Count', index=False)
volume.to_excel(writer, sheet_name='GMV and GPV', index=False)
better_together.to_excel(writer, sheet_name='Better Together', index=False)
writer.save()

In [30]:
arr

,Month,Month Start,Month End,Live Core Payments ARR,Live Guest Payments ARR,Live Payments ARR,Live Core SaaS ARR,Live Guest SFDC SaaS ARR,Live Rev Share ARR,Live EC ARR,Live XC ARR,Live SaaS ARR,Live ARR
0,April - 2022,2022-04-01,2022-04-17,3.052868e+08,4.888688e+07,3.541737e+08,1.914921e+08,7.508818e+07,5639073.36,2.696428e+07,6882776.16,3.060664e+08,6.602401e+08
1,March - 2022,2022-03-01,2022-03-31,2.855298e+08,4.770667e+07,3.332365e+08,1.871720e+08,7.309317e+07,9642836.76,2.660498e+07,6770371.68,3.032834e+08,6.365199e+08
2,February - 2022,2022-02-01,2022-02-28,2.568750e+08,4.791571e+07,3.047907e+08,1.780728e+08,6.865607e+07,7858243.44,2.374708e+07,6938495.76,2.852726e+08,5.900633e+08
3,January - 2022,2022-01-01,2022-01-31,2.442237e+08,4.725328e+07,2.914770e+08,1.713158e+08,6.498613e+07,7958948.52,2.233540e+07,6368854.80,2.729652e+08,5.644422e+08
4,December - 2021,2021-12-01,2021-12-31,2.542143e+08,4.817194e+07,3.023862e+08,1.681373e+08,6.275580e+07,6938719.32,2.108668e+07,6256898.40,2.651754e+08,5.675616e+08
5,November - 2021,2021-11-01,2021-11-30,2.567074e+08,4.624784e+07,3.029552e+08,1.628617e+08,5.941889e+07,5962192.56,2.054874e+07,5418671.76,2.542102e+08,5.571654e+08
6,October - 2021,2021-10-01,2021-10-31,2.647574e+08,4.774387e+07,3.125012e+08,1.562775e+08,5.496110e+07,5758666.68,2.006901e+07,5045880.00,2.421121e+08,5.546134e+08
7,September - 2021,2021-09-01,2021-09-30,2.693868e+08,4.817721e+07,3.175640e+08,1.504906e+08,5.049472e+07,1695362.40,1.893188e+07,4607400.00,2.262200e+08,5.437840e+08
8,August - 2021,2021-08-01,2021-08-31,2.683790e+08,4.926815e+07,3.176471e+08,1.446629e+08,4.703723e+07,1732376.16,1.800310e+07,4248000.00,2.156836e+08,5.333308e+08
9,July - 2021,2021-07-01,2021-07-31,2.641320e+08,5.154311e+07,3.156751e+08,1.394335e+08,4.396208e+07,1734990.84,1.692848e+07,4020000.00,2.060790e+08,5.217541e+08
